# 이터러블 객체
반복을 위하 정의한 로직을 사용해 자체 이터러블 구현  
**이터러블**: &#95;iter&#95;&#95; 매직 메서드를 구현한 객체  
**이터레이터**: &#95;&#95;next&#95;&#95;매직 메서드를 구현한 객체  

#### 이터러블 프로토콜
- 파이썬의 자체 프로토콜로 이터러블 프로토콜을 이용해 파이썬의 반복이 동작함
```python
for e in myobject:
```

위의 형태로 반복할 수 있는지 확인하기 위해 파이썬은 다음 두 가지를 차례대로 확인함
- 객체가 &#95;&#95;next&#95;&#95;나 &#95;&#95;iter&#95;&#95;이터레이터 메서드 중 하나를 포함하는지 여부
- 객체가 시퀀스이고 __len__과 __getitem__를 모두 가졌는지 여부

<hr>

### 이터러블 객체 만들기
객체를 반복하려고 하면 파이썬은 해당 객체의 iter() 함수를 호출  
iter() 함수가 처음으로 하는 일은 해당 객체에 &#95;&#95;iter&#95;&#95; 메서드가 있는 지를 확인하고 있다면 &#95;&#95;iter&#95;&#95; 메서드를 실행  

In [5]:
# 일정 기간의 날짜를 하루 간격으롱 반복하는 객체의 코드

from datetime import timedelta

class DateRangeIterable:
    """자체 이터레이터 메서드를 가지고 있는 이터러블"""
    
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

In [6]:
from datetime import date

for day in DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5)):
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


__위의 코드에서는__
1. 파이썬이 iter() 함수 호출
1. iter() 함수는 &#95;&#95;iter&#95;&#95; 매직 메서드 호출
1. &#95;&#95;iter&#95;&#95; 메서드는 self를 반환하고 있으므로 객체 자신이 이터러블임을 나타냄 => 루프의 각 단계에서마다 자신의 next() 함수를 호출
1. next() 함수는 다시  &#95;&#95;next&#95;&#95; 메서드에 위임
1. &#95;&#95;next&#95;&#95; 메서드는 요소를 어떻게 생산하고 하나씩 반환할 것인지 결정
1. 더이상 생산할 것이 없을 경우 파이썬에게 StopIteration 예외를 발생시켜 전달  

__for 루프의 작동 원리는 StopIteration이 발생할 때까지 next()를 호출하는 것__

In [11]:
r = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
print(next(r))
print(next(r))
print(next(r))
print(next(r))
print(next(r))

2019-01-01
2019-01-02
2019-01-03
2019-01-04


StopIteration: 

위의 코드의 경우 한 번 실행하면 끝의 날짜에 도달한 상태이므로 이후에 호출하면 계속 StopIteration 예외가 발생  
두 번 이상 반복하고자 하면 첫번째 반복에서만 정상 작동하고 두번째부터는 작동하지 않음  

아래의 예제에서 볼 수 있 듯 두번째 max에서는 에러가 발생함

In [12]:
r1 = DateRangeIterable(date(2019, 1, 1), date(2019, 1, 5))
print(", ".join(map(str, r1)))
max(r1)

2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04


ValueError: max() arg is an empty sequence

문제 발생의 이유: __반복 프로토콜의 작동 방식__ 때문
- 이터러블 객체는 이터레이터를 생성하고 이를 이용해 반복을 수행
- 위에서 &#95;&#95;iter&#95;&#95;는 self를 반환했으나 호출될 때마다 새로운 이터레이터를 만들 수 있음

In [13]:
class DateRangeContainerIterable:
    """자체 이터레이터 메서드를 가지고 있는 이터러블"""
    
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
                
    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)

In [16]:
r1 = DateRangeContainerIterable(date(2019, 1, 1), date(2019, 1, 5))
print(", ".join(map(str, r1)))
print("max value:", max(r1))

2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04
max value: 2019-01-04


위의 코드에서 for 루프는 &#95;&#95;iter&#95;&#95;를 호출하고 &#95;&#95;iter&#95;&#95;는 다시 제너레이터를 생성함  
이와 같은 형태의 객체를 __컨테이너 이터러블(container iterable)__이라고 한다

___
*참고*  
yield 키워드는 제너레이터를 반환할 때 사용됨  
파이썬에서 제너레이터는 여러 개의 데이터를 미리 만들어 놓지 않고 필요할 때마다 즉석에서 하나씩 만들어낼 수 있는 객체를 의미

# 시퀀스 만들기
객체에 &#95;&#95;iter&#95;&#95; 메서드를 정의하지 않았지만 반복하기를 원하는 경우도 있는데, iter() 함수는 객체에 &#95;&#95;iter&#95;&#95;가 정의되어 있지 않으면 &#95;&#95;getitme&#95;&#95;을 찾고 없으면 TypeError를 발생시킴  


위의 경우는 이터러블을 사용하므로 한 번에 하나의 날짜만 보관하여 메모리를 적게 사용하나 n번째 요소를 얻기 위한 시간 복잡도는 O(n)이다.   
시퀀스로 구현하게 된다면 더 많은 메모리가 사용되지만 (모든 것을 한 번에 보관) 특정 요소를 가져오기 위한 인덱싱의 시간 복잡도는 O(1)이다.

In [20]:
class DateRangeSequence:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()
        
    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days
    
    def __getitem__(self, day_no):
        return self._range[day_no]
    
    def __len__(self):
        return len(self._range)

In [21]:
s1 = DateRangeSequence(date(2019, 1, 1), date(2019, 1, 5))
for day in s1:
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04


아래와 같이 음수 인덱스도 동작

In [23]:
print(s1[0])
print(s1[-1])
print(s1[3])

2019-01-01
2019-01-04
2019-01-04
